In [29]:
import pymongo
import pandas as pd

In [30]:
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [39]:
def toGeoJSON(lista):
    return {
            "type": "Point",
            "coordinates":[lista[1],lista[0]]
        }

def getValuatio(funding_round):
    valor = 0
    for lista in funding_round:
        valor = lista['raised_amount']
    return valor

    


def convertRestaurant(company):
    lista_dict = []
    for index in range(len(company['offices'])):
        for key in company['offices']:
            if key['latitude'] != None and key['longitude'] != None:
                comp_off = {
                "name": company["name"],
                "category_code": company["category_code"],
                "number_of_employees": company["number_of_employees"],
                "founded_year": company["founded_year"],
                "valuation": getValuatio(company['funding_rounds']),
                "position": toGeoJSON([key['latitude'],key['longitude']])   
                }
                lista_dict.append(comp_off)
        return lista_dict

cursor = db.companies.find({"$and": [ { "offices.latitude": {"$exists": True} },{ "offices.longitude": {"$exists": True} }, { "founded_year": {"$gt": 1995} }, {"deadpooled_year": None}  ] },{"_id":0})
ofiComp = list(map(lambda r: convertRestaurant(r),cursor))
geoCompanies = [element for lista in ofiComp for element in lista]


In [42]:
df = pd.DataFrame(geoCompanies)

df.head()
   


category_code           object
founded_year             int64
name                    object
number_of_employees    float64
position                object
valuation              float64
dtype: object

In [55]:
def getRatingCode(code):
    
    ratingCode = 0
    if code == ("web" or "software" or "hardware" or "design" or "consulting"):
        ratingCode += 5
    elif code == ("advertising" or "games_video" or  "ecommerce"):
        ratingCode += 4
    elif code == ("network_hosting" or "search" or "cleantech" or "analytics" or "messaging"):
        ratingCode += 3
    elif code == ("mobile" or "public_relations"  or "finance" or "fashion"):
        ratingCode += 2
    elif code == ("biotec" or "security" or "semiconductor" or "education" or "medical" or "health" or "manufacturing" or "nanotech" or "social" or "music" or "news" or "travel" or "photo_video"):    
        ratingCode += 1
    else:
        ratingCode += 0
    return ratingCode

ratingCode = df['category_code'].map(getRatingCode)
    


In [54]:
def getRatingFunds(funds):
    ratingFunds = 0
    if funds >= 100000000:
        ratingFunds += 5
    elif funds >= 8000000 and funds < 100000000:
        ratingFunds += 4
    elif funds >= 3000000 and funds < 8000000:
        ratingFunds += 3
    elif funds >= 1500000 and funds < 3000000:
        ratingFunds += 2
    elif funds >= 500000 and funds < 1500000:
        ratingFunds += 1
    else:
        ratingFunds += 0
    return ratingFunds


ratingFunds = df['valuation'].map(getRatingFunds)    


In [ ]:
def getRatingEmpl(employees):
    ratingEmpl = 0
    if employees >= 50000:
        ratingEmpl += 5
    elif employees >= 10000 and employees < 50000:
        ratingEmpl += 4
    elif employees >= 1000 and employees < 10000:    
        ratingEmpl += 3
    elif employees >= 200 and employees < 1000:    
        ratingEmpl += 2
    elif employees >= 50 and employees < 200:    
        ratingEmpl += 1
    else:    
        ratingEmpl += 0
    return ratingEmpl

ratingEmpl = df['number_of_employees'].map(getRatingEmpl)   

    
    


In [60]:
def getRatingYear(year):        
    ratingYear = 0
    if year > 2010:
        ratingYear += 5
    return ratingYear

ratingYear = df['founded_year'].map(getRatingYear)


In [62]:
def rating(df):
    return df['number_of_employees'].map(getRatingEmpl) + df['valuation'].map(getRatingFunds) + df['category_code'].map(getRatingCode) + df['founded_year'].map(getRatingYear)

rating = rating(df)

In [63]:
df['rating'] = rating

In [64]:
df.head()

,category_code,founded_year,name,number_of_employees,position,valuation,rating
0,news,2004,Digg,60.0,"{'type': 'Point', 'coordinates': [-122.394523,...",5000000.0,4
1,web,1999,Postini,NaN,"{'type': 'Point', 'coordinates': [-122.247573,...",0.0,5
2,web,2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",5000000.0,8
3,web,2003,Gizmoz,NaN,"{'type': 'Point', 'coordinates': [-122.169472,...",5300000.0,8
4,web,2002,StumbleUpon,NaN,"{'type': 'Point', 'coordinates': [-122.419204,...",NaN,5


In [65]:
df.to_json("comp_with_coords.json", orient="records", lines=True)

In [113]:
companies = list(db.location.find({}))
df = pd.DataFrame(companies)

In [114]:
def getAreasRat(companylist):
    areasRating = []
    for i in range(len(companylist)):    
        compRadio = db.location.find({
          "position": {
            "$near": {
              "$geometry": {
                "type": "Point",
                "coordinates": companylist[i]['position']['coordinates']
              },
              "$minDistance": 0,
              "$maxDistance":10000
            }
          }
        })
        areaRating = sum([i['rating'] for i in list(compRadio)])   
        areasRating.append(areaRating)
    return areasRating
  

In [116]:
listOfRating = getAreasRat(companies)

In [119]:
df['area_rating'] = listOfRating
df.head()

,_id,category_code,founded_year,name,number_of_employees,position,rating,valuation,area_rating
0,5c6a6d800c548a0a1e03935d,news,2004,Digg,60.0,"{'type': 'Point', 'coordinates': [-122.394523,...",4,5000000.0,1746
1,5c6a6d800c548a0a1e03935e,web,1999,Postini,NaN,"{'type': 'Point', 'coordinates': [-122.247573,...",5,0.0,861
2,5c6a6d800c548a0a1e03935f,web,2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",8,5000000.0,337
3,5c6a6d800c548a0a1e039360,web,2003,Gizmoz,NaN,"{'type': 'Point', 'coordinates': [-122.169472,...",8,5300000.0,930
4,5c6a6d800c548a0a1e039361,web,2002,StumbleUpon,NaN,"{'type': 'Point', 'coordinates': [-122.419204,...",5,NaN,1741


In [137]:
coords_df = pd.DataFrame(pd.DataFrame(df["position"].values.flatten().tolist())["coordinates"].values.tolist())
clean_df = pd.concat([df,coords_df],axis=1)
clean_df.drop(['_id',"position"], inplace=True, axis=1)
clean_df = clean_df.rename({0:"long",1:"lat"}, axis=1)
clean_df.head()

,category_code,founded_year,name,number_of_employees,rating,valuation,area_rating,long,lat
0,news,2004,Digg,60.0,4,5000000.0,1746,-122.394523,37.764726
1,web,1999,Postini,NaN,5,0.0,861,-122.247573,37.506885
2,web,2006,Geni,18.0,8,5000000.0,337,-118.393064,34.090368
3,web,2003,Gizmoz,NaN,8,5300000.0,930,-122.169472,37.484130
4,web,2002,StumbleUpon,NaN,5,NaN,1741,-122.419204,37.775196


In [143]:
selected_comp = clean_df.loc[clean_df['area_rating'].idxmax()]
selected_name = selected_comp['name']
selected_coord = [selected_comp['long'],selected_comp['lat']]
display(selected_name)
display(selected_coord)

'Brightstorm'

[-122.371203, 37.804619]

In [144]:
clean_df.to_json('visualize_location.json', orient="records")